In [1]:
import os
import pandas as pd
from Dataset import Dataset
from DataLoader import DataLoader
from Speech import Speech
from preprocessors import *
from utils import scrape_speeches
from tqdm import tqdm
import json

/home/m_hovo/Desktop/KU Leuven/Modern Data Analytics/mda-project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [2]:
usr_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
cwd = os.getcwd()
path = f"{cwd}/dataset_typical"

In [3]:
if not os.path.exists(path) or len(os.listdir(path)) == 0:
    scrape_speeches("https://www.americanrhetoric.com/", path, usr_agent)

In [4]:
preprocessors = [
    RemoveExtras(names=True),
    RemoveStopwords(),
    Lemmatize(),
    RemovePunctuation(),
    Lower()
]
dataset = Dataset(path, preprocessors)
dataloader = DataLoader(dataset)

In [5]:
visual_words = pd.read_csv("results/visual_words.csv", header=None)
visual_words = list(visual_words[0].array)

In [6]:
results_dir = f"{cwd}/results"
imagery_words_proportion_saving_dir = f"{cwd}/results/imagery_words_proportion_typical.csv"
imagery_words_proportion = {
    "count": [],
    "proportion_in_speech": []
}

In [7]:
with tqdm(total=len(dataloader.dataset)) as progress_bar:
    for i, speech_content in enumerate(dataloader):
        speech = Speech(speech_content)
        imagery_words_proportion["count"].append(speech.get_proportion_in_speech(visual_words, absolute=True))
        imagery_words_proportion["proportion_in_speech"].append(speech.get_proportion_in_speech(visual_words, absolute=False))
        progress_bar.update(1)

100%|██████████| 77/77 [05:14<00:00,  4.08s/it]


In [14]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
with open(imagery_words_proportion_saving_dir, "w") as write_file:
    json.dump(imagery_words_proportion, write_file, indent=4)